In [1]:
###Importing Important Libraries

import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import os
import random

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization,GlobalMaxPooling2D
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
###Unzipping the Data
import zipfile

##Train
local_zip_train = '/kaggle/input/dogs-vs-cats/train.zip'
zip_ref = zipfile.ZipFile(local_zip_train, 'r')
zip_ref.extractall('/kaggle/working')
zip_ref.close()

##Test
local_zip_test = '/kaggle/input/dogs-vs-cats/test1.zip'
zip_ref = zipfile.ZipFile(local_zip_test, 'r')
zip_ref.extractall('/kaggle/working')
zip_ref.close()

In [3]:
###Image Hyperparameters

IMAGE_SIZE = 224
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE,IMAGE_SIZE
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

In [4]:

###Preparing Training Data

filenames = os.listdir("/kaggle/working/train")
print(len(filenames))

categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append('dog')
    else:
        categories.append('cat')

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

25000


In [5]:
df.head()

,filename,category
0,dog.1787.jpg,dog
1,dog.9656.jpg,dog
2,dog.2435.jpg,dog
3,cat.4758.jpg,cat
4,dog.5430.jpg,dog


In [6]:
df.tail()

,filename,category
24995,dog.4927.jpg,dog
24996,cat.12390.jpg,cat
24997,cat.1278.jpg,cat
24998,dog.11691.jpg,dog
24999,cat.5452.jpg,cat


In [7]:
###Creating the Model
from keras.applications import VGG16

input_shape = (IMAGE_HEIGHT,IMAGE_WIDTH,IMAGE_CHANNELS)

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")

for layer in pre_trained_model.layers[:15]:
    layer.trainable = False
for layer in pre_trained_model.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

58892288/58889256 [==============================] - 3s 0us/step


In [8]:
from keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

In [9]:

###Callbacks

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

earlystop = EarlyStopping(patience=10)

callbacks = [earlystop, learning_rate_reduction]

In [10]:

###Preparing the Data

train_df, validate_df = train_test_split(df, test_size=0.10, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [11]:
###Hyperparameters


total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size= 16
epochs = 5

In [12]:
###Generator

##Train
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/kaggle/working/train", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=batch_size
)


Found 22500 validated image filenames belonging to 2 classes.


In [13]:
##Val 
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/kaggle/working/train", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='binary',
    batch_size=batch_size
)

Found 2500 validated image filenames belonging to 2 classes.


In [14]:
###Training the Model

history = model.fit_generator(
    train_generator, 
    epochs=5,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

Epoch 1/5
1406/1406 [==============================] - 316s 224ms/step - loss: 0.1687 - accuracy: 0.9278 - val_loss: 0.1773 - val_accuracy: 0.9647
Epoch 2/5
   1/1406 [..............................] - ETA: 1:09 - loss: 0.1201 - accuracy: 0.9375

/opt/conda/lib/python3.6/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1406/1406 [==============================] - 311s 221ms/step - loss: 0.1022 - accuracy: 0.9578 - val_loss: 0.0154 - val_accuracy: 0.9722
Epoch 3/5
1406/1406 [==============================] - 305s 217ms/step - loss: 0.0805 - accuracy: 0.9680 - val_loss: 0.0164 - val_accuracy: 0.9710racy: 
Epoch 4/5
1406/1406 [==============================] - 300s 213ms/step - loss: 0.0676 - accuracy: 0.9721 - val_loss: 0.0164 - val_accuracy: 0.9775
Epoch 5/5
1406/1406 [==============================] - 299s 213ms/step - loss: 0.0594 - accuracy: 0.9759 - val_loss: 0.0196 - val_accuracy: 0.9758
